# Extract and Transform Parking Violations Data

#### In this notebook we will be extracting information from an API route and save a csv file in order to build it into our sqlite database.

1. Data source from PPA (Philadelphia Parking Authority): [OpenDataPhilly](https://www.opendataphilly.org/dataset/parking-violations)
2. API Documentation: [API Explorer](https://cityofphiladelphia.github.io/carto-api-explorer/#parking_violations)

In [1]:
import pandas as pd
import numpy as np
import requests
# import json
import random
from pprint import pprint

In [2]:
# Create our url parameters. We will get all data from 2017
datetime_begin = '2017-01-01'
datetime_end = '2018-01-01'

base_url = 'https://phl.carto.com/api/v2/sql?q=SELECT * FROM parking_violations'
datetime_filter = f"WHERE issue_datetime >= '{datetime_begin}' AND issue_datetime < '{datetime_end}'"

In [3]:
# Build our query url
query_url = f'{base_url} {datetime_filter}'
print(query_url)

https://phl.carto.com/api/v2/sql?q=SELECT * FROM parking_violations WHERE issue_datetime >= '2017-01-01' AND issue_datetime < '2018-01-01'


In [4]:
# Request data
response = requests.get(query_url)
data = response.json()
rows = data['rows']
print(response)

<Response [200]>


In [5]:
# pprint(rows)

In [6]:
# Check the size of the data
len(rows)

1722620

In [7]:
# Save the desired fields from the response json into our df columns

# Lists to hold column values to be inserted into the df
anon_ticket_number = []
issue_datetime = []
state = []
anon_plate_id = []
location = []
violation_desc = []
fine = []
issuing_agency = []
lat = []
lon = []
zip_code = []

# Set a counter to track progress since the data is large...
counter = 0

for row in rows:
     
    counter += 1
    if counter % 500000 == 0:
        print(f'Retrieving Results for row #{counter}.')

    try:
        anon_ticket_number.append(row['anon_ticket_number'])
        issue_datetime.append(row['issue_datetime'])
        state.append(row['state'])
        anon_plate_id.append(row['anon_plate_id'])
        location.append(row['location'])
        violation_desc.append(row['violation_desc'])
        fine.append(row['fine'])
        issuing_agency.append(row['issuing_agency'])
        lat.append(row['lat'])
        lon.append(row['lon'])
        zip_code.append(row['zip_code'])
    except (KeyError, IndexError):
        print('Missing field/result... skipping.')
    
print('------------------------')
print('------------------------')
print('Completed! All records successfully saved in the dataframe!')

ppa_df = pd.DataFrame({'anon_ticket_number': anon_ticket_number,
                   'issue_datetime': issue_datetime,
                   'state': state,
                   'anon_plate_id': anon_plate_id,
                   'location': location,
                   'violation_desc': violation_desc,
                   'fine': fine,
                   'issuing_agency': issuing_agency,
                   'lat': lat,
                   'lon': lon,
                   'zip_code': zip_code
})

Retrieving Results for row #500000.
Retrieving Results for row #1000000.
Retrieving Results for row #1500000.
------------------------
------------------------
Completed! All records successfully saved in the dataframe!


In [8]:
ppa_df.head()

,anon_ticket_number,issue_datetime,state,anon_plate_id,location,violation_desc,fine,issuing_agency,lat,lon,zip_code
0,8979623,2017-12-06T12:29:00Z,PA,4728925,6053 CASTOR AVE,BUS ONLY ZONE,51,PPA,40.035503,-75.081110,19149
1,8979624,2017-10-16T18:03:00Z,PA,4728926,4300 CRESSON ST,STOPPING PROHIBITED,51,PPA,40.025712,-75.222495,19127
2,8979625,2017-11-02T22:09:00Z,PA,4728926,4300 CRESSON ST,OVER TIME LIMIT,26,PPA,40.025792,-75.222557,19127
3,8979626,2017-11-05T20:19:00Z,PA,4728926,4300 CRESSON ST,OVER TIME LIMIT,26,PPA,40.025898,-75.222708,19127
4,8979627,2017-11-11T13:53:00Z,NJ,4728927,200 N 11TH ST,METER EXPIRED CC,36,PPA,39.956063,-75.157375,19107


In [9]:
# Check if there's any null values and how many
ppa_df.isna().sum()

anon_ticket_number        0
issue_datetime            0
state                     0
anon_plate_id             0
location                  0
violation_desc            0
fine                      0
issuing_agency            0
lat                   67660
lon                   67658
zip_code                  0
dtype: int64

##### There are a small number of rows without geo coordinates. We're comfortable to remove those rows as it's a small percentage compared with the overall dataset.

In [10]:
# Drop NaN
ppa_df.dropna(axis=0, how='any', inplace=True)

In [11]:
# Check if there's any hidden null values (e.g. empty strings '' or all spaces '  ') and replace them with NaN if any
ppa_df = ppa_df.replace(r'^\s*$', np.nan, regex=True)

In [12]:
# Check null values again and if there are more we will likely to drop them
ppa_df.isna().sum()

anon_ticket_number         0
issue_datetime             0
state                      4
anon_plate_id              0
location                  98
violation_desc             0
fine                       0
issuing_agency             1
lat                        0
lon                        0
zip_code              221187
dtype: int64

##### Again a small number of rows have null values. We will drop them

In [13]:
# Drop additional NaN
cleaned_df = ppa_df.dropna(axis=0, how='any')
# Check row count again...
print(len(cleaned_df))

1433770


##### Make sure the data types are what we need them to be

In [14]:
# Check datatypes
cleaned_df.dtypes

anon_ticket_number      int64
issue_datetime         object
state                  object
anon_plate_id          object
location               object
violation_desc         object
fine                    int64
issuing_agency         object
lat                   float64
lon                   float64
zip_code               object
dtype: object

In [15]:
# Convert issue_datetime from a string to a datetime object
cleaned_df['issue_datetime'] = pd.to_datetime(cleaned_df['issue_datetime'])
# Check converstion successfulness
cleaned_df['issue_datetime'].dtype

/Users/ZTC/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


datetime64[ns, UTC]

In [16]:
# Remove timezone since this is strictly Phillly data 
cleaned_df['issue_datetime'] = cleaned_df['issue_datetime'].dt.tz_localize(None)
cleaned_df.head()

/Users/ZTC/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,anon_ticket_number,issue_datetime,state,anon_plate_id,location,violation_desc,fine,issuing_agency,lat,lon,zip_code
0,8979623,2017-12-06 12:29:00,PA,4728925,6053 CASTOR AVE,BUS ONLY ZONE,51,PPA,40.035503,-75.081110,19149
1,8979624,2017-10-16 18:03:00,PA,4728926,4300 CRESSON ST,STOPPING PROHIBITED,51,PPA,40.025712,-75.222495,19127
2,8979625,2017-11-02 22:09:00,PA,4728926,4300 CRESSON ST,OVER TIME LIMIT,26,PPA,40.025792,-75.222557,19127
3,8979626,2017-11-05 20:19:00,PA,4728926,4300 CRESSON ST,OVER TIME LIMIT,26,PPA,40.025898,-75.222708,19127
4,8979627,2017-11-11 13:53:00,NJ,4728927,200 N 11TH ST,METER EXPIRED CC,36,PPA,39.956063,-75.157375,19107


In [17]:
cleaned_df.dtypes

anon_ticket_number             int64
issue_datetime        datetime64[ns]
state                         object
anon_plate_id                 object
location                      object
violation_desc                object
fine                           int64
issuing_agency                object
lat                          float64
lon                          float64
zip_code                      object
dtype: object

### Since there is too much data this dataset, we will randomly select only a portion of it for the sake of this project. The rationale is to reduce load time for testing purposes and to shrink the size of the database to be uploaded to Github...

In [104]:
copied_df = cleaned_df.copy()
copied_df['month'] = cleaned_df['issue_datetime'].dt.month

##### In order to do this, we wanted to check how evenly the data is distributed over the months

In [105]:
copied_df['month'].value_counts()

8     132872
10    131070
9     125881
11    125406
5     124456
3     122807
6     120453
4     118885
1     110035
7     109191
2     108253
12    104461
Name: month, dtype: int64

In [106]:
# Since it looks like the data is distributed pretty evenly throughout the year, we decided to do a random
# selection on the entire dataset. We will use pandas to sample 30% of our cleaned data.
reduced_df = copied_df.sample(frac=0.3)

In [107]:
# Check data distribution again
reduced_df['month'].value_counts()

8     39676
10    39166
9     37715
11    37606
5     37463
3     36727
6     36069
4     35884
1     33096
7     32881
2     32455
12    31393
Name: month, dtype: int64

##### More datetime manipulations and add additional columns for ease of use in sqlalchemy later

In [108]:
# Add a day column
reduced_df['day'] = reduced_df['issue_datetime'].dt.day
# Add an hour column
reduced_df['hour'] = reduced_df['issue_datetime'].dt.hour

In [109]:
# New datetime column used for parsing. Will be deleted later.
reduced_df['issue_datetime_2'] = reduced_df['issue_datetime']
reduced_df['issue_datetime_2'] = reduced_df['issue_datetime_2'].astype(str)

In [110]:
# Add a year-month-day-hour identifier column. This will come in handy for our our analysis such as scatter plot
ymdh = []

for datetime in reduced_df['issue_datetime_2']:
    ymdh.append(datetime.split(':')[0])
    
reduced_df['ymdh'] = ymdh

In [111]:
reduced_df = reduced_df.drop(columns='issue_datetime_2')

In [112]:
# Check data types again
reduced_df.dtypes

anon_ticket_number             int64
issue_datetime        datetime64[ns]
state                         object
anon_plate_id                 object
location                      object
violation_desc                object
fine                           int64
issuing_agency                object
lat                          float64
lon                          float64
zip_code                      object
month                          int64
day                            int64
hour                           int64
ymdh                          object
dtype: object

In [113]:
# Sort by datetime and reset index
final_df = reduced_df.sort_values(by='issue_datetime', ascending=True)
final_df.reset_index(drop=True, inplace=True)

In [114]:
# Inspect
final_df

,anon_ticket_number,issue_datetime,state,anon_plate_id,location,violation_desc,fine,issuing_agency,lat,lon,zip_code,month,day,hour,ymdh
0,7695773,2017-01-01 00:00:00,PA,4340382,3131 N 7TH ST,HP RESERVED SPACE,301,POLICE,39.999414,-75.142332,19133,1,1,0,2017-01-01 00
1,7915128,2017-01-01 00:30:00,NJ,4420623,1747 N 17TH ST,PARKING PROHBITED,41,POLICE,39.980069,-75.162530,19121,1,1,0,2017-01-01 00
2,7806422,2017-01-01 01:00:00,PA,2418180,3100 STOUTON ST,SIDEWALK,51,POLICE,39.994725,-75.114731,19134,1,1,1,2017-01-01 01
3,7727051,2017-01-01 01:45:00,PA,1581609,4100 N 5TH ST,DOUBLE PARKED,51,POLICE,40.014261,-75.135362,19140,1,1,1,2017-01-01 01
4,8053935,2017-01-01 02:05:00,PA,1545498,4800 RORER ST,CORNER CLEARANCE,51,POLICE,40.022871,-75.114553,19120,1,1,2,2017-01-01 02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430126,9137143,2017-12-31 15:21:00,PA,2177729,3900 WALNUT ST,EXPIRED INSPECTION,41,PPA,39.953947,-75.200687,19104,12,31,15,2017-12-31 15
430127,9026154,2017-12-31 15:23:00,IN,4755706,500 SOUTH ST,STOP PROHIBITED CC,76,PPA,39.942040,-75.152243,19147,12,31,15,2017-12-31 15
430128,9034759,2017-12-31 15:26:00,NY,4760419,300 CHESTNUT ST,METER EXPIRED CC,36,PPA,39.948892,-75.147330,19106,12,31,15,2017-12-31 15
430129,9271232,2017-12-31 15:32:00,DE,4878557,400 SOUTH ST,METER EXPIRED CC,36,PPA,39.941738,-75.150227,19147,12,31,15,2017-12-31 15


In [115]:
# Export dataset before reduce for reference to CSV
# cleaned_df.to_csv('../resources/cleaned_data/cleaned_parking_violations_2017.csv', index=False)

In [116]:
# Export dataset after reduce to CSV 
final_df.to_csv('../resources/cleaned_data/cleaned_reduced_parking_violations_2017.csv', index=False)